In [7]:
!pip install pillow scipy

  Using cached https://files.pythonhosted.org/packages/e6/37/fc5bde88462cf8a069e522dd6a0d259b69faba1326f0a3ce8b4e11d9f317/Pillow-6.0.0-cp36-cp36m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/b9/a2/62f77d2d3c42364d45ba714b4bdf7e1c4dfa67091dc9f614fa5a948b4fb4/scipy-1.2.1-cp36-cp36m-win_amd64.whl


In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os, shutil

In [2]:
base_dir = 'datasets\\dogs-vs-cats'

original_dir = os.path.join(base_dir, 'original')
original_train_dir = os.path.join(original_dir, 'train')

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
#test_dir = os.path.join(base_dir, 'test')

cats_train_dir = os.path.join(train_dir, 'cats')
cats_validation_dir = os.path.join(validation_dir, 'cats')
#cats_test_dir = os.path.join(test_dir, 'cats')

dogs_train_dir = os.path.join(train_dir, 'dogs')
dogs_validation_dir = os.path.join(validation_dir, 'dogs')
#dogs_test_dir = os.path.join(test_dir, 'dogs')

In [9]:
for d in [train_dir, validation_dir, #test_dir, 
          cats_train_dir, cats_validation_dir, #cats_test_dir, 
          dogs_train_dir, dogs_validation_dir, #dogs_test_dir
         ]:
    if not os.path.exists(d):
        os.makedirs(d)

In [10]:
def copy_images_to_folder(filename_pattern, start_range, stop_range, src_dir, dst_dir):
    filenames = [filename_pattern.format(i) for i in range(start_range, stop_range)]
    for filename in filenames:
        src = os.path.join(src_dir, filename)
        dst = os.path.join(dst_dir, filename)
        shutil.copyfile(src, dst)
        
copy_images_to_folder('cat.{}.jpg', 0, 10000, original_train_dir, cats_train_dir)
copy_images_to_folder('cat.{}.jpg', 10000, 12500, original_train_dir, cats_validation_dir)

copy_images_to_folder('dog.{}.jpg', 0, 10000, original_train_dir, dogs_train_dir)
copy_images_to_folder('dog.{}.jpg', 10000, 12500, original_train_dir, dogs_validation_dir)

In [3]:
batch_size = 20

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(50,50), 
                                                    batch_size=batch_size, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=(50,50),
                                                              batch_size=batch_size, class_mode='binary')
#test_generator = test_datagen.flow_from_directory(test_dir, target_size=(50,50),
#                                                  batch_size=batch_size, class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
inputs = tf.keras.Input(shape=(50,50,3))
x = layers.Conv2D(32, (3,3), activation=tf.nn.relu)(inputs)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64, (3,3), activation=tf.nn.relu)(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(128, (3,3), activation=tf.nn.relu)(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(128, (3,3), activation=tf.nn.relu)(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(512, activation=tf.nn.relu)(x)
outputs = layers.Dense(1, activation=tf.nn.sigmoid)(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0     

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
model.fit_generator(train_generator, 
                    steps_per_epoch=20000/batch_size, 
                    epochs=10, 
                    validation_data=validation_generator, 
                    validation_steps=5000/batch_size)

Epoch 1/10
1000/1000 [==============================] - 124s 124ms/step - loss: 0.6582 - accuracy: 0.5967 - val_loss: 0.6106 - val_accuracy: 0.6508
Epoch 2/10
1000/1000 [==============================] - 141s 141ms/step - loss: 0.5312 - accuracy: 0.7351 - val_loss: 0.4815 - val_accuracy: 0.7614: 7s - los
Epoch 3/10
1000/1000 [==============================] - 148s 148ms/step - loss: 0.4523 - accuracy: 0.7875 - val_loss: 0.4706 - val_accuracy: 0.7870 20s - loss: 0.4569 - acc - ETA: 19s - ETA: 6s - l - ETA - ETA: 1s - loss: 0.4525 - accuracy: 0.78 - ETA: 1s - loss:
Epoch 4/10
1000/1000 [==============================] - 142s 142ms/step - loss: 0.3952 - accuracy: 0.8186 - val_loss: 0.4223 - val_accuracy: 0.8066
Epoch 5/10
1000/1000 [==============================] - 141s 141ms/step - loss: 0.3446 - accuracy: 0.8479 - val_loss: 0.3985 - val_accuracy: 0.8200
Epoch 6/10
1000/1000 [==============================] - 147s 147ms/step - loss: 0.3015 - accuracy: 0.8698 - val_loss: 0.3794 - val_acc